In [1]:
import json
import gcsfs
# dir = C:\Users\annab\Desktop\ArgoEKE\github-argo-eke\data\
fp = r"C:\Users\annab\Desktop\ArgoEKE\pangeo-181919-e7bc5bdaf4d5.json" # dir+r'pangeo-181919-e7bc5bdaf4d5.json' #
with open(fp) as f:
    token = json.load(f)
gcs = gcsfs.GCSFileSystem(token=token) # file system instance
bucket_pth = 'pangeo-argo-eke/'

In [3]:
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
plt.rcParams['figure.dpi'] = 350
import xarray as xr
import numpy as np
import os
import datetime
import pandas as pd

# set the cwd to be the parent directory 
os.chdir(os.path.dirname(os.getcwd()))
print('Current working directory: ', os.getcwd())

Current working directory:  c:\Users\annab\Desktop\ArgoEKE


Open NOAA drifter-based climatology dataset

In [4]:
# gcs.mkdir('pangeo-argo-eke/climatology/drifter/ ')
gcs.ls('pangeo-argo-eke/climatology/drifter')
# gcs.ls()

['pangeo-argo-eke/climatology/drifter/drifter_monthlymeans.nc',
 'pangeo-argo-eke/climatology/drifter/drifter_variance.nc']

In [3]:
# local_path_drifter_var= os.path.join(os.getcwd(), 'drifter_variance.nc')
# local_path_drifter_means= os.path.join(os.getcwd(), 'drifter_monthlymeans.nc')
# remote_path_drifter_var = bucket_pth+ 'climatology/drifter/drifter_variance.nc'
# remote_path_drifter_means = bucket_pth+ 'climatology/drifter/drifter_monthlymeans.nc'
# print(local_path_drifter_means, remote_path_drifter_means)
# print(local_path_drifter_var, remote_path_drifter_var)

# gcs.put(local_path_drifter_var, remote_path_drifter_var, recursive=True)
# gcs.put(local_path_drifter_means, remote_path_drifter_means, recursive=True)

In [ ]:
drifter_ds = xr.open_dataset(os.path.join(os.getcwd(), 'drifter_monthlymeans.nc'), decode_times=False)
    # gcs.get_mapper('pangeo-argo-eke/climatology/drifter/drifter_monthlymeans.nc'))
drifter_ds

<xarray.Dataset> Size: 532MB
Dimensions:  (latitude: 632, longitude: 1440, time: 12)
Coordinates:
    Lat      (latitude) float64 5kB ...
    Lon      (longitude) float64 12kB ...
    Time     (time) float64 96B ...
Dimensions without coordinates: latitude, longitude, time
Data variables:
    U        (time, longitude, latitude) float64 87MB ...
    V        (time, longitude, latitude) float64 87MB ...
    SST      (time, longitude, latitude) float64 87MB ...
    eU       (time, longitude, latitude) float64 87MB ...
    eV       (time, longitude, latitude) float64 87MB ...
    eSST     (time, longitude, latitude) float64 87MB ...
    N        (longitude, latitude) float64 7MB ...
Attributes:
    title:        drifter_monthlymeans
    description:  Monthly Means of Drifter Data
    version:      3.10
    date2:        File created 21-Aug-2023 09:38:21
    datadate:     Climatology uses data through 28-Feb-2023
    institution:  NOAA/AOML & Univ. Miami
    author:       Laurindo, Mariano, and Lumpkin (2017), Deep-Sea Res. I, doi...

Open Argo float-based geostrophic velocity (climatology) dataset

In [ ]:
plevel=2
argo_ds_10dbar = xr.open_zarr(gcs.get_mapper( 'pangeo-argo-eke/climatology/version_1/plevel'+str(plevel)+'.zarr'), consolidated=True)
argo_ds_10dbar

<xarray.Dataset> Size: 398MB
Dimensions:    (latitude: 720, longitude: 1440, month: 12, pressure: 1)
Coordinates:
  * latitude   (latitude) float64 6kB -90.0 -89.75 -89.5 ... 89.25 89.5 89.75
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * month      (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * pressure   (pressure) float64 8B 10.0
Data variables:
    u          (latitude, longitude, month, pressure) float64 100MB dask.array<chunksize=(720, 1440, 1, 1), meta=np.ndarray>
    umse       (latitude, longitude, month, pressure) float64 100MB dask.array<chunksize=(720, 1440, 1, 1), meta=np.ndarray>
    v          (latitude, longitude, month, pressure) float64 100MB dask.array<chunksize=(720, 1440, 1, 1), meta=np.ndarray>
    vmse       (latitude, longitude, month, pressure) float64 100MB dask.array<chunksize=(720, 1440, 1, 1), meta=np.ndarray>
Attributes:
    created_date:     2025-02-02 11:03:56.003185
    description:      Absolute geostrophic velocity estimate monthly climatol...
    largescale_file:  pangeo-argo-eke/testfolder_112/vel_largescale_monthly_g...
    method:           DataArray.groupby("time.month").mean()
    smallscale_file:  pangeo-argo-eke/testfolder_127/vel_smallscale_monthly_g...

In [8]:
plevel=3
argo_ds_20dbar = xr.open_zarr(gcs.get_mapper( 'pangeo-argo-eke/climatology/version_1/plevel'+str(plevel)+'.zarr'), consolidated=True)
# argo_ds_20dbar

Match the spatial locations of the two datasets

In [ ]:
print(ds)

In [ ]:
# subsample the Argo dataset (0.25 degree grid) to match the locations of the drifter dataset (1 degree grid)

def subsample_argo_to_drifter(argo_ds, drifter_ds):
    """
    Subsample the Argo dataset to match the locations of the drifter dataset.
    """
    argo_ds_subsampled = argo_ds.sel(
        latitude=drifter_ds.latitude,
        longitude=drifter_ds.longitude,
        method='nearest'
    )
    return argo_ds_subsampled